Scrap websites mentioned in file to collect food categories. It saves scrapped data after each page. 

In [34]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
import time
from tqdm import tqdm

## Read file

Sometimes there is an error if you don't to specified number of rows 'nrows' to read from a file. If it happens check first how many rows are there in the scv file.

In [57]:
pathToRead = 'C:/Users/Patrycja/Desktop/Uni Koblenz/CSS/GermansFoodPreferences/scrapper/files/jan/data_2.csv'

In [59]:
df = pd.read_csv(pathToRead, sep=',', encoding='ISO-8859-1')#, nrows=nrows)#, engine='python'

In [16]:
df.head()

,country,country_region,zip,city,date,target_link,origin_link,host_type,recipe_id
0,DE,16,-1,Berlin,2013-07-28 13:59:59,/rezept/404417/Knuspriger-Spanferkelrollbraten...,http://www.kochbar.de/rezepte/spanferkelrollbr...,Mozilla/5.0 (Windows NT 5.1; rv:22.0) Gecko/20...,404417
1,DE,16,-1,Berlin,2013-07-28 13:59:59,/rezept/404417/Knuspriger-Spanferkelrollbraten...,http://www.kochbar.de/rezepte/spanferkelrollbr...,Mozilla/5.0 (Windows NT 5.1; rv:22.0) Gecko/20...,404417
2,DE,4,22117,Hamburg,2013-07-02 04:06:39,/rezept/106165/Spargelsalat-mit-Rinderfilet-un...,http://www.google.de/url?sa=t&rct=j&q=salat%20...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:21.0) G...,106165
3,DE,5,-1,Frankfurt_Am_Main,2013-07-14 10:00:38,/rezept/470972/Nudelsalat-mit-Gemuese.html,-,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,470972
4,DE,16,14169,Berlin,2013-07-22 16:00:17,/rezept/417178/Ravioli-gefuellt-mit-Pilzen.html,-,Mozilla/5.0 (iPad; CPU OS 6_1_3 like Mac OS X)...,417178


## Collect data

In [41]:
def find_comment(soup, wanted_comment='REZEPT-KATEGORIEN'):
    found = False

    comments = soup.find_all(string=lambda text:isinstance(text, Comment))
    i = 0
    while not found and i < len(comments):
        node = comments[i]
        if node.strip() == wanted_comment:
            #print('comment found')
            found = True

        i+=1
    return node

In [42]:
def find_categories(soup, wanted_class = 'rtli-pb-small'):
    data = []
    node = find_comment(soup)
    startOfItems, endOfItems = False, False
    
    while not endOfItems and node.find_next_sibling():
        node = node.find_next_sibling()

        if node['class'][0] == wanted_class:
            startOfItems = True

            if node.findChild().name == 'a':
                data.append(node.a.text)
            elif node.findChild().name == 'span':
                data.append(node.span.text)

        elif startOfItems:
            endOfItems = True
    
    return data

In [43]:
domain = 'http://www.kochbar.de'

Remember to specify with rows to process. If 'all_rows' is set to True, 'start_index' and 'end_index' is not taken into account.

In [44]:
start_index, end_index = 110, 120
all_rows = True

In [45]:
def listToString(listToConvert):
    result = '['
    for el in listToConvert:
        result += el + ','
    
    result = result[:-1]
    return result +']'

In [46]:
def improveText(text):
    text = text.replace('ê','e')
    text = text.replace('Ê','E')
    
    text = text.replace('ö','oe')
    text = text.replace('Ö','Oe')
    text = text.replace('ä','ae')
    text = text.replace('Ä','Ae')
    text = text.replace('ü','ue')
    text = text.replace('Ü','Ue')
    text = text.replace('ß','ss')

    for i in range(len(text)):
        nr = ord(text[i])
        if (nr >= 97 and nr <= 122) or (nr >=65 and nr <=90) or nr in [32, 34, 38, 39, 40, 41, 44, 45, 46, 47, 63, 91, 92, 93, 95]:
            pass
        else:
            print('to replace:', text[i])
            text = text.replace(text[i], '?')
                
    return text

In [47]:
pathToSave = 'scrapped_data.csv'

In [ ]:
start = time.time()

error_num = 0
df_new = pd.DataFrame(columns=['date', 'recipe_id', 'country', 'region', 'city', 'categories'])
f= open(pathToSave,'w')
df_new.to_csv(f, index=None, header=True, encoding='utf-8', sep=',')
f.close()

df = pd.read_csv(pathToRead, sep=',', encoding='ISO-8859-1') #, nrows=nrows)#, engine='python'

f= open(pathToSave, 'a')

if not all_rows:
    df = df[start_index:end_index]
else:
    start_index = 0

with tqdm(total=nrows) as pbar:
    for i in range(len(df)): 
        #print(domain + url)
        url = df['target_link'][start_index + i]
        response = requests.get(domain + url)

        if response.url != 'https://www.kochbar.de/rezepte/' and response.status_code == 200:
            #print('Request status ok!')
            soup = BeautifulSoup(response.content.decode('utf-8','ignore'), "lxml")
            categories = find_categories(soup)
            #print('categories:', categories, '\n')
            #data.append(categories)

            s = pd.DataFrame({'d': str(df['date'][start_index + i]), 'id': df['recipe_id'][start_index + i], 'country':df['country'][start_index + i], 'region':df['country_region'][start_index + i], 'city': improveText(df['city'][start_index + i]),'categories': improveText(str(categories))}, index=[0])
        elif response.status_code == 200:
            #print('This webpage is probably no longer available!')
            #data.append(None)
            s = pd.DataFrame({'d': df['date'][start_index + i], 'id': df['recipe_id'][start_index + i], 'country':df['country'][start_index + i], 'region':df['country_region'][start_index + i], 'city': improveText(df['city'][start_index + i]), 'categories': str([])}, index=[0])

        else:
            #print('Not correct request status: ', response.status_code)
            s = pd.DataFrame({'d': df['date'][start_index + i], 'id': df['recipe_id'][start_index + i], 'country':df['country'][start_index + i], 'region':df['country_region'][start_index + i], 'city': improveText(df['city'][start_index + i]), 'categories': str([])}, index=[0])
        try:
            s.to_csv(f, index=None, header=False, encoding='utf-8', sep=',')
        except UnicodeEncodeError:
            print('UnicodeEncodeError')
            print(s)
            error_num += 1

        pbar.update(1)

f.close() 
print(time.time() - start)
print('errors:', error_num)

In [33]:
f.close() 

## Check saved data

In [ ]:
df1=pd.read_csv(pathToSave)
df1

In [358]:
df1['categories'][1][2:-2].replace("'","").replace('"','').split(',')

['Europa',
 ' Fisch',
 ' Mittagstisch',
 ' Abendessen',
 ' Date',
 ' Fruehling',
 ' Sommer',
 ' Herbst',
 ' Winter',
 ' Hauptspeise',
 ' Gesund und Diaet',
 ' preiswerte',
 ' schnell & einfach',
 ' Italien',
 ' fettarm',
 ' glutenfrei',
 ' ohne Weizen']